In [36]:
import pandas as pd
from pandasql import sqldf
import numpy as np
import os

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, Normalizer
from sklearn.metrics import accuracy_score

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Sequential, layers, callbacks, metrics, optimizers

from kerastuner.tuners import RandomSearch

from xgboost import XGBModel

In [37]:
os.listdir()

['gender_submission.csv',
 'Untitled.ipynb',
 'my_submission.csv',
 'train.csv',
 'test.csv',
 'titanic',
 'tmp',
 'export',
 '.ipynb_checkpoints']

In [38]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [39]:
def get_title(v):
    
    rares = ['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']
    miss = ['Mlle', 'Ms']
    mrs = ['Mme']
    
    try:
        after_comma = v.split(',')[1].split()
        if after_comma[0]=='the':
            returning_variable = after_comma[1][0:-1]
        else:
            returning_variable = after_comma[0][0:-1]
        
        if returning_variable in rares:
            return 'Rare'
        elif returning_variable in miss:
            return 'Miss'
        elif returning_variable in mrs:
            return 'Mrs'
        else:
            return returning_variable
    except:
        return None

In [40]:
train['Title'] = train['Name'].apply(get_title)
test['Title'] = test['Name'].apply(get_title)

In [6]:
# cols = ['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
label = ['Survived']
features = ['Age', 'SibSp', 'Parch', 'Fare', 'Title']

In [7]:
submission = test[['PassengerId']]
mean_age = train.Age.mean()
train.loc[train.Age.isna(), 'Age'] = mean_age
test.loc[test.Age.isna(), 'Age'] = mean_age

In [8]:
mean_fare = train.Fare.mean()
test.loc[test.Fare.isna(), 'Fare'] = mean_fare

In [9]:
train = train.loc[~train.Embarked.isna()].copy()

In [10]:
class_labeller = LabelEncoder()
sex_labeller = LabelEncoder()
port_labeller = LabelEncoder()
title_labeller = LabelEncoder()

In [11]:
train.loc[:, 'Pclass']   = class_labeller.fit_transform(train['Pclass'])
train.loc[:, 'Sex']      = sex_labeller.fit_transform(train['Sex'])
train.loc[:, 'Embarked'] = port_labeller.fit_transform(train['Embarked'])
train.loc[:, 'Title']    = title_labeller.fit_transform(train['Title'])

test.loc[:, 'Pclass']   = class_labeller.transform(test['Pclass'])
test.loc[:, 'Sex']      = sex_labeller.transform(test['Sex'])
test.loc[:, 'Embarked'] = port_labeller.transform(test['Embarked'])
test.loc[:, 'Title']    = title_labeller.fit_transform(test['Title'])

In [12]:
train_classes = pd.get_dummies(train.Pclass, prefix='pclass')
train_sexes = pd.get_dummies(train.Sex, prefix='sex')
train_embarks = pd.get_dummies(train.Embarked, prefix='embarked')
train_titles = pd.get_dummies(train.Title, prefix='title')

test_classes = pd.get_dummies(test.Pclass, prefix='pclass')
test_sexes = pd.get_dummies(test.Sex, prefix='sex')
test_embarks = pd.get_dummies(test.Embarked, prefix='embarked')
test_titles = pd.get_dummies(test.Title, prefix='title')

In [13]:
train = pd.concat([train[label], train[features], train_classes, train_sexes, train_embarks, train_titles], axis=1)
test = pd.concat([test[features], test_classes, test_sexes, test_embarks, test_titles], axis=1)

In [14]:
y_train, x_train = np.array(train[train.columns[0]]), np.array(train[train.columns[1:]])
x_test = np.array(test)

In [15]:
normalizer = Normalizer()

In [16]:
x_train = normalizer.fit_transform(x_train)
x_test = normalizer.transform(x_test)

# XGBoost

In [17]:
xgb = XGBModel()

In [18]:
xgb.fit(x_train, y_train)

[22:08:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBModel(base_score=0.5, booster='gbtree', colsample_bylevel=1,
     colsample_bynode=1, colsample_bytree=1, gamma=0,
     importance_type='gain', learning_rate=0.1, max_delta_step=0,
     max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
     n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
     reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
     subsample=1, verbosity=1)

In [19]:
preds = xgb.predict(x_test)

# Keras

In [57]:
def build_model(hp):
    model = Sequential()
    model.add(layers.Dense(units=hp.Range('units_0',
                                          min_value=32,
                                          max_value=512,
                                          step=32),
                           activation='relu', input_shape=(18,)))
    model.add(layers.Dropout(hp.Choice('dropout_rate_0', values=[0, 0.1, 0.2, 0.4])))
    model.add(layers.Dense(units=hp.Range('units_1',
                                          min_value=32,
                                          max_value=512,
                                          step=32),
                           activation='relu'))
    model.add(layers.Dropout(hp.Choice('dropout_rate_0', values=[0, 0.1, 0.2, 0.4])))
    model.add(layers.Dense(units=hp.Range('units_2',
                                          min_value=32,
                                          max_value=512,
                                          step=32),
                           activation='relu'))
    model.add(layers.Dropout(hp.Choice('dropout_rate_1', values=[0, 0.1, 0.2, 0.4])))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer=optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [58]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='',
    project_name='titanic')

In [59]:
tuner.search_space_summary()

In [60]:
w1 = y_train.sum()/y_train.shape[0]
w0 = 1-w1

early = callbacks.EarlyStopping(monitor='val_binary_accuracy', patience=3, restore_best_weights=True)

tuner.search(x_train, 
          y_train, 
          epochs=100, 
          class_weight={0: w0, 1:w1}, 
          validation_split=0.20,
          callbacks=[early])

W0728 22:27:00.845973 140647803262784 callbacks.py:1249] Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


W0728 22:27:00.933765 140647803262784 callbacks.py:1249] Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


W0728 22:27:01.022345 140647803262784 callbacks.py:1249] Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


W0728 22:27:01.108412 140647803262784 callbacks.py:1249] Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


W0728 22:27:01.193296 140647803262784 callbacks.py:1249] Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


...

W0728 22:30:31.176515 140647803262784 callbacks.py:1249] Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


W0728 22:30:31.348062 140647803262784 callbacks.py:1249] Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


W0728 22:30:31.520684 140647803262784 callbacks.py:1249] Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


W0728 22:30:31.692938 140647803262784 callbacks.py:1249] Early stopping conditioned on metric `val_binary_accuracy` which is not available. Available metrics are: loss,accuracy,val_loss,val_accuracy


Name,Best model,Current model
accuracy,0.8359,0.7675
loss,0.1778,0.2349
val_loss,0.1481,0.1896
val_accuracy,0.8764,0.8352


In [61]:
tuner.results_summary()

In [62]:
model = tuner.get_best_models()[0]

In [63]:
tuner.hyperparameters.values

{'units_0': 96,
 'dropout_rate_0': 0.2,
 'units_1': 288,
 'units_2': 288,
 'dropout_rate_1': 0.4,
 'learning_rate': 0.0001}

In [64]:
model.evaluate(x_train, y_train)

889/889 [==============================] - ETA: 0s - loss: 0.4651 - accuracy: 0.78 - 0s 82us/sample - loss: 0.3912 - accuracy: 0.8436


[0.39116318711614984, 0.84364456]

In [65]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 192)               3648      
_________________________________________________________________
dropout_3 (Dropout)          (None, 192)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 352)               67936     
_________________________________________________________________
dropout_4 (Dropout)          (None, 352)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                11296     
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [66]:
for l in model.layers:
    print('==============================')
    print(l.get_config())
print('==============================')

{'name': 'dense_4', 'trainable': True, 'batch_input_shape': (None, 18), 'dtype': 'float32', 'units': 192, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dropout_3', 'trainable': True, 'dtype': 'float32', 'rate': 0.1, 'noise_shape': None, 'seed': None}
{'name': 'dense_5', 'trainable': True, 'dtype': 'float32', 'units': 352, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'dropout_4', 'trainable': True, 'dtype': 'float32', 'rate': 0.1, 'noise

In [67]:
submission['Survived'] = model.predict_classes(x_test)
submission.to_csv('my_submission.csv', index=False)